# NoteBook train model

## Import thư viện

In [1]:
import pickle
import numpy as np
import cv2
import os

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Dense, MaxPool2D, Conv2D, BatchNormalization, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


## Preprocess data

Xử lý dữ liệu thô

In [ ]:
# Lấy thông tin các tệp con (trùng tên với label) trong tệp data_raw_image
list_label = os.listdir('data_image_raw')

# List lưu ảnh đã mã hóa và label tương ứng
data_img = []
label = []

for item in list_label:
    # Tạo dường dẫn đến từng tệp con (có thể dùng cộng xâu bth) cách bên dưới an toàn hơn
    path_label = os.path.join('data_image_raw', item)
    list_image = os.listdir(path_label) # Trả về tên các tệp ảnh
    
    for image in list_image:        # Tạo đường dẫn đến thư mục ảnh
        path_image = os.path.join(path_label, image)
        
        # Đọc ảnh
        matrix = cv2.imread(path_image)
        
        # Thêm ảnh và label tương ứng vào các list
        data_img.append(matrix)
        label.append(item)

    # In ra màn thông báo
    print(f'Đã xử lý xong ảnh của: {item} với số ảnh: {len(list_image)}')

# Chuyển data và label về np.array vì tensorflow yêu cầu đầu vào là np.array, label cần đưa về dạng 2D
data_img = np.array(data_img) 
cat_label = set(label.copy())
label = np.array(label).reshape(-1, 1) # Có thể dùng expand_dim cũng được

# Hiển thị ra màn console
print(f'Shape của data: {data_img.shape} với các label {cat_label}')

Đã xử lý xong ảnh của: Hang với số ảnh: 500
Đã xử lý xong ảnh của: Tuan với số ảnh: 500
Đã xử lý xong ảnh của: Viet Anh với số ảnh: 500
Shape của data: (1500, 128, 128, 3) với các label {'Hang', 'Viet Anh', 'Tuan'}


Lấy data từ file thư mục chứa 

In [3]:
# Encoder label
encoder = OneHotEncoder(sparse_output= False)
label_processed = encoder.fit_transform(label)
# Chuẩn hóa data đầu vào
data_processed = data_img.astype('float32') / 255

In [4]:
data_processed.shape

(1500, 128, 128, 3)

Lưu categories lại vào file để tiện dùng lại để gán label

In [5]:
with open('model/categories.pkl', 'wb') as f:
    pickle.dump(encoder.categories_, f)

In [6]:
with open('model/categories.pkl', 'rb') as f:
    cat = pickle.load(f)

lb = np.array(cat[0])

num_class = lb.size # Số class dùng để phân biệt

Chia tập train test

In [7]:

xtrain, xtest, ytrain, ytest = train_test_split(data_processed, label_processed, test_size= 0.2)

## Tạo model và train

In [8]:
# from tensorflow.keras.applications import VGG16

In [9]:


# # Load VGG16 without the top layers
# vgg = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# # Freeze the pretrained layers to retain pre-trained weights
# vgg.trainable = False

# # Build the new model
# model_cnn = Sequential([
#     # VGG as feature extractor
#     vgg,
#     Flatten(),  # Flatten the output of VGG16

#     # Fully connected layers
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(num_class, activation='softmax'),
# ])
# model_cnn.compile(optimizer= Adam(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['acc', 'precision', 'recall'])

# model_cnn.summary() 

Tiến hành train model

In [10]:
model_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)), # Yêu cầu input_shape phải là 3D
    MaxPool2D((2, 2), padding = 'same'),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPool2D((2, 2), padding = 'same'),
    
    Flatten(),
    
    Dense(128, activation='relu'),
    Dropout(0.5),
    
    Dense(num_class, activation= 'softmax'),
])
model_cnn.compile(optimizer= Adam(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['acc', 'precision', 'recall'])

model_cnn.summary() 

C:\Users\levie\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 31, 31, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 61504)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     7,872,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,892,419 (30.11 MB)

 Trainable params: 7,892,419 (30.11 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model_cnn.fit(xtrain, ytrain, epochs= 10, validation_data = (xtest, ytest) , batch_size= 32)

Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 231ms/step - acc: 0.6477 - loss: 0.8411 - precision: 0.7036 - recall: 0.5899 - val_acc: 0.9967 - val_loss: 0.0207 - val_precision: 0.9967 - val_recall: 0.9967
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 259ms/step - acc: 0.9965 - loss: 0.0251 - precision: 0.9967 - recall: 0.9952 - val_acc: 0.9967 - val_loss: 0.0091 - val_precision: 0.9967 - val_recall: 0.9967
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 9s 243ms/step - acc: 0.9970 - loss: 0.0164 - precision: 0.9970 - recall: 0.9964 - val_acc: 1.0000 - val_loss: 4.9230e-04 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 9s 229ms/step - acc: 1.0000 - loss: 0.0045 - precision: 1.0000 - recall: 0.9996 - val_acc: 1.0000 - val_loss: 8.0317e-04 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 9s 230ms/step - acc: 1.0000 - loss: 0.0030 - precision: 1.0000 - recall: 1.0000 - val_acc: 1.0000 - val_loss: 8.3338e-05 - val_precision: 1.0000 - val_r

## Lưu model lại

In [12]:
model_cnn.save('model/model_cnn.h5', include_optimizer = True) 